In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Wed May  3 10:16:06 2017

@author: zhouyi
"""
import glob
import os
import sys

import implicit
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.sparse

import evaluation


DATA_ROOT = '/Users/zhouyi/Documents/GraduatePJ/steam_data'

In [2]:
unique_uid = list()
with open(os.path.join(DATA_ROOT, 'unique_user_id.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())
        
unique_gname = list()
with open(os.path.join(DATA_ROOT, 'unique_game_name.txt'), 'r') as f:
    for line in f:
        unique_gname.append(line.strip())

n_games = len(unique_gname)
n_users = len(unique_uid)

In [3]:
# initialize a implicit model
IMSTEAMmodel = implicit.als.AlternatingLeastSquares(factors=50)

In [4]:
#Load the data and train the model
#To fit the  implicit Arg requirements, rows are item, cols are user
'''implicit.fit:
     item_users (csr_matrix): Matrix of confidences for the liked items. This matrix
                should be a csr_matrix where the rows of the matrix are the
                item, the columns are the users that liked that item, and the
                value is the confidence that the user liked the item.'''
                
def load_data(csv_file, shape=(n_games, n_users)):
    st = pd.read_csv(csv_file)
    cols, rows = np.array(st['user_id'], dtype=np.int32), np.array(st['game_name'], dtype=np.int32)
    count = st['value']
    return scipy.sparse.csr_matrix((count,(rows, cols)), dtype=np.int16, shape=shape), rows, cols

In [5]:
train_data, rows, cols = load_data(os.path.join(DATA_ROOT, 'train.num.sub.csv'))
print 'train_data.shape:',train_data.shape
print 'train_data entries:',train_data.data.shape

vad_data, rows_vad, cols_vad = load_data(os.path.join(DATA_ROOT, 'vad.num.sub.csv'))
print 'vad__data.shape:',vad_data.shape
print 'vad_data entries:',vad_data.data.shape

test_data, rows_test, cols_test = load_data(os.path.join(DATA_ROOT, 'test.num.sub.csv'))
print 'test_data.shape:',test_data.shape
print 'test_data entries:',test_data.data.shape

print 'sparse according to vad_data:',float(vad_data.data.shape[0])/(vad_data.shape[0]*vad_data.shape[1])
print '---'


train_data.shape: (1197, 1269)
train_data entries: (54310,)
vad__data.shape: (1197, 1269)
vad_data entries: (6065,)
test_data.shape: (1197, 1269)
test_data entries: (15152,)
sparse according to vad_data: 0.00399277679357
---


In [6]:
#fit the WMF
IMSTEAMmodel.fit(train_data.astype(np.double))

In [8]:
'''Compute validation metric '''
def validate(model, X, vad_data):
        
    vad_recall_at_k = evaluation.Recall_at_k(X.T, vad_data.T,
                                                model.user_factors,
                                                model.item_factors,
                                                k=20)
        
    vad_map_at_k = evaluation.MAP_at_k(X.T, vad_data.T,
                                                model.user_factors,
                                                model.item_factors,
                                                k=20)
        
    vad_ndcg_at_k = evaluation.NDCG_at_k(X.T, vad_data.T,
                                                model.user_factors,
                                                model.item_factors,
                                                k=20)
    if True:
        print('\tNDCG@k: %.4f' % vad_ndcg_at_k)
        print('\tRecall@k: %.4f' % vad_recall_at_k)
        print('\tMAP@k: %.4f' % vad_map_at_k)
        sys.stdout.flush()
            
        
    return vad_ndcg_at_k,vad_recall_at_k,vad_map_at_k

print 'Validation Results for WMF:'
VAD_RECORD=validate(IMSTEAMmodel,train_data,vad_data)

Validation Results for WMF:
	NDCG@k: 0.0239
	Recall@k: 0.1825
	MAP@k: 0.0197


In [9]:
print 'Test Results for WMF:'
TEST_RECORD=validate(IMSTEAMmodel,train_data,test_data)

Test Results for WMF:
	NDCG@k: 0.0593
	Recall@k: 0.2033
	MAP@k: 0.0512
